In [23]:
import requests
import pandas as pd
import time
#使用pandas保存数据
pd.set_option('max_rows',500)

headers={
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.85 Safari/537.36'
}

url='https://c.m.163.com/ug/api/wuhan/app/data/list-total'
r=requests.get(url,headers=headers)

In [24]:
print(r.status_code)
print(type(r.text))
print(len(r.text))

200
<class 'str'>
300164


In [30]:
import json
data_json=json.loads(r.text)
data_json.keys()

dict_keys(['reqId', 'code', 'msg', 'data', 'timestamp'])

In [32]:
data=data_json['data']
data.keys()

dict_keys(['chinaTotal', 'chinaDayList', 'lastUpdateTime', 'overseaLastUpdateTime', 'areaTree'])

# 爬取全国各省实时数据

In [41]:
china_data=data['areaTree'][2]
print(china_data['name'])
print(china_data.keys())

中国
dict_keys(['today', 'total', 'extData', 'name', 'id', 'lastUpdateTime', 'children'])


In [63]:
data_province=data['areaTree'][2]['children']
print('children是一个list,每个元素结构相同： '+str(type(data_province)))
print('data_province[0].keys(): '+str(data_province[0].keys()))
print("data_province[0]['today']"+str(data_province[0]['today'].keys()))
print("data_province[0]['total']"+str(data_province[0]['total'].keys()))

children是一个list,每个元素结构相同： <class 'list'>
data_province[0].keys(): dict_keys(['today', 'total', 'extData', 'name', 'id', 'lastUpdateTime', 'children'])
data_province[0]['today']dict_keys(['confirm', 'suspect', 'heal', 'dead', 'severe', 'storeConfirm'])
data_province[0]['total']dict_keys(['confirm', 'suspect', 'heal', 'dead', 'severe', 'input'])


In [65]:
pd.DataFrame(data_province).head()

,today,total,extData,name,id,lastUpdateTime,children
0,"{'confirm': 0, 'suspect': None, 'heal': 0, 'de...","{'confirm': 68157, 'suspect': 0, 'heal': 63640...",{},湖北,420000,2021-04-23 08:37:16,"[{'today': {'confirm': 0, 'suspect': None, 'he..."
1,"{'confirm': 14, 'suspect': None, 'heal': 14, '...","{'confirm': 11718, 'suspect': 0, 'heal': 11316...",{},香港,810000,2021-04-23 09:12:15,"[{'today': {'confirm': 14, 'suspect': None, 'h..."
2,"{'confirm': 5, 'suspect': None, 'heal': 0, 'de...","{'confirm': 2327, 'suspect': 0, 'heal': 2273, ...",{},广东,440000,2021-04-23 09:12:15,"[{'today': {'confirm': 5, 'suspect': None, 'he..."
3,"{'confirm': 5, 'suspect': None, 'heal': 4, 'de...","{'confirm': 1955, 'suspect': 0, 'heal': 1889, ...",{},上海,310000,2021-04-23 08:37:16,"[{'today': {'confirm': 5, 'suspect': None, 'he..."
4,"{'confirm': 0, 'suspect': None, 'heal': 0, 'de...","{'confirm': 1610, 'suspect': 0, 'heal': 1597, ...",{},黑龙江,230000,2021-04-23 00:02:08,"[{'today': {'confirm': 0, 'suspect': None, 'he..."


     data_province为多层嵌套字典的list， 
    可以将其拆成四个部分，（today）（total）（extData）（id，name，lastUpdateTime）， 
    分别处理，利用dataFrame生成表格，再拼接起来


In [68]:
info=pd.DataFrame(data_province)[['id','lastUpdateTime','name']]
info.head()

,id,lastUpdateTime,name
0,420000,2021-04-23 08:37:16,湖北
1,810000,2021-04-23 09:12:15,香港
2,440000,2021-04-23 09:12:15,广东
3,310000,2021-04-23 08:37:16,上海
4,230000,2021-04-23 00:02:08,黑龙江


In [70]:
today_data=pd.DataFrame([province['today'] for province in data_province ])
today_data.head()

,confirm,suspect,heal,dead,severe,storeConfirm
0,0,None,0,0,None,0
1,14,None,14,0,None,0
2,5,None,0,0,None,5
3,5,None,4,0,None,1
4,0,None,0,0,None,0


In [73]:
today_data.columns=['today_'+i for i in today_data.columns]
today_data.head()


,today_today_today_confirm,today_today_today_suspect,today_today_today_heal,today_today_today_dead,today_today_today_severe,today_today_today_storeConfirm
0,0,None,0,0,None,0
1,14,None,14,0,None,0
2,5,None,0,0,None,5
3,5,None,4,0,None,1
4,0,None,0,0,None,0


In [111]:
total_data=pd.DataFrame([province['total'] for province in data_province])
total_data.drop(['newConfirm','newDead','newHeal'],axis = 1,inplace = True)
total_data.columns=['total_'+i for i in total_data.columns]
total_data

,total_confirm,total_suspect,total_heal,total_dead,total_severe,total_input
0,68157,0,63640,4512,0,0
1,11727,0,11324,209,0,0
2,2327,0,2273,8,0,0
3,1955,0,1889,7,0,0
4,1610,0,1597,13,0,0
5,1331,0,1320,1,0,0
6,1317,0,1310,7,0,0
7,1312,0,1285,22,0,0
8,1090,0,1044,11,0,0
9,1057,0,1038,9,0,0


In [80]:
ext_data=pd.DataFrame([province['extData'] for province in data_province])
ext_data

""
0
1
2
3
4
5
6
7
8
9


In [83]:
pd.concat([info,total_data,today_data],axis=1)

,id,lastUpdateTime,name,total_confirm,total_suspect,total_heal,total_dead,total_severe,total_input,total_newConfirm,total_newDead,total_newHeal,today_today_today_confirm,today_today_today_suspect,today_today_today_heal,today_today_today_dead,today_today_today_severe,today_today_today_storeConfirm
0,420000,2021-04-23 08:37:16,湖北,68157,0,63640,4512,0,0,NaN,NaN,NaN,0,None,0,0,None,0
1,810000,2021-04-23 09:12:15,香港,11718,0,11316,209,0,0,NaN,NaN,NaN,14,None,14,0,None,0
2,440000,2021-04-23 09:12:15,广东,2327,0,2273,8,0,0,NaN,NaN,NaN,5,None,0,0,None,5
3,310000,2021-04-23 08:37:16,上海,1955,0,1889,7,0,0,NaN,NaN,NaN,5,None,4,0,None,1
4,230000,2021-04-23 00:02:08,黑龙江,1610,0,1597,13,0,0,NaN,NaN,NaN,0,None,0,0,None,0
5,330000,2021-04-23 09:27:13,浙江,1331,0,1320,1,0,0,NaN,NaN,NaN,2,None,0,0,None,2
6,130000,2021-04-23 00:02:08,河北,1317,0,1310,7,0,0,NaN,NaN,NaN,0,None,0,0,None,0
7,410000,2021-04-23 00:02:08,河南,1312,0,1285,22,0,0,NaN,NaN,NaN,0,None,0,0,None,0
8,700000,2021-04-23 09:12:14,台湾,1086,0,1041,11,0,0,NaN,NaN,NaN,4,None,3,0,None,1
9,110000,2021-04-23 00:02:09,北京,1057,0,1038,9,0,0,463.0,0.0,453.0,0,None,0,0,None,0


# 将上述提取数据的过程封装成函数

In [121]:
def get_data(data,info_list):
    info=pd.DataFrame(data)[info_list] #主要信息
    
    today_data=pd.DataFrame([i['today'] for i in data])         #生成today的数据 
    today_data.columns=['today_'+i for i in today_data.columns] #修改列名
    
    total_data=pd.DataFrame([i['total'] for i in data]) #生成total的数据
    total_data.drop(['newConfirm','newDead','newHeal'],axis = 1,inplace = True)
    total_data.columns=['total_'+i for i in total_data.columns] #修改列名
    
    return pd.concat([info,total_data,today_data],axis=1)

In [122]:
import json
import requests
import pandas as pd
import time
from openpyxl import load_workbook
#使用pandas保存数据
pd.set_option('max_rows',500)

headers={
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.85 Safari/537.36'
}

url='https://c.m.163.com/ug/api/wuhan/app/data/list-total'
r=requests.get(url,headers=headers)
data_json=json.loads(r.text)
data=data_json['data']
data_province=data['areaTree'][2]['children']
cur_province=get_data(data_province,['id','lastUpdateTime','name'])
type(cur_province)

pandas.core.frame.DataFrame

In [123]:
#将处abs理好的数据写入excel中
outputpath=r'C:\Users\YouZiming\Desktop\DATABASE\大三春学习资料\软件工程\团队项目\cur_province.xlsx'
#writer = pd.ExcelWriter(outputpath,engine='openpyxl')#可以向不同的sheet写入数据
#book=load_workbook(outputpath)
#writer.book = book
cur_province.to_excel(outputpath,sheet_name='data')
#writer.save()

# 待完成的工作
1. 只爬取了实时的全国各省的新冠肺炎疫情情况，即不涉及全球、各地级市，或者疫苗情况
2. 目前只能运行脚本爬取，将结果存入excel文件中。不能定时爬取，文件处理的细节有待完善。
3. 数据源为网易新闻：https://wp.m.163.com/163/page/news/virus_report/index.html?_nw_=1&_anw_=1
   由于数据格式问题，只支持该数据源